In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to get and parse HTML content from a Wikipedia page
def get_html_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Function to extract the article title
def extract_title(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    title = soup.find('h1', {'id': 'firstHeading'}).text
    return title

# Function to extract article text with headings
def extract_text_with_headings(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    content = soup.find('div', {'id': 'bodyContent'})
    
    data = {}
    current_heading = ""
    
    for tag in content.find_all(['h2', 'h3', 'h4', 'p']):
        if tag.name in ['h2', 'h3', 'h4']:
            current_heading = tag.text.strip()
            data[current_heading] = []
        elif tag.name == 'p':
            if current_heading in data:
                data[current_heading].append(tag.text.strip())
            else:
                data[current_heading] = [tag.text.strip()]
    
    return data

# Function to collect every link that redirects to another Wikipedia page
def collect_internal_links(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    content = soup.find('div', {'id': 'bodyContent'})
    
    links = []
    for link in content.find_all('a', href=True):
        href = link['href']
        if href.startswith('/wiki/') and not href.startswith('/wiki/Special:'):
            links.append('https://fr.wikipedia.org' + href)
    
    return links

# Function to wrap all previous functions into one
def scrape_wikipedia_page(url):
    html_content = get_html_content(url)
    if html_content is None:
        return None
    
    title = extract_title(html_content)
    text_with_headings = extract_text_with_headings(html_content)
    internal_links = collect_internal_links(html_content)
    
    return {
        'title': title,
        'text_with_headings': text_with_headings,
        'internal_links': internal_links
    }

# Test the function on the specified Wikipedia page
if __name__ == '__main__':
    url = 'https://fr.wikipedia.org/wiki/Dell'
    result = scrape_wikipedia_page(url)
    
    if result:
        # Print the results for verification
        print("Title:", result['title'])
        
        print("\nText with Headings:")
        for heading, paragraphs in result['text_with_headings'].items():
            print(f"\n{heading}")
            for paragraph in paragraphs:
                print(paragraph)
        
        print("\nInternal Links:")
        for link in result['internal_links']:
            print(link)
    else:
        print("Failed to retrieve content.")
